In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os, json, re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging

/opt/conda/envs/wordle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from wordle import Wordle
from prompting import Prompt_Helper
game = Wordle()
p = Prompt_Helper()

In [4]:
# Config model + quantization
llama3_8b = "meta-llama/Meta-Llama-3-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# Load in NF4 to fit 15GB GPU
tokenizer = AutoTokenizer.from_pretrained(llama3_8b, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(llama3_8b, 
                                             quantization_config=nf4_config,
                                             device_map="auto", )
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [02:08<00:00, 32.05s/it]


In [5]:
conversation = p.get_few_shot_instructions(2)
conversation

[{'role': 'system',
  'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)},
 {'role': 'system', 'content': 'Example #1:'},
 {'role': 'user', 'content': 'Enter guess #0'},
 {'role': 'assistant', 'content': 'BAGEL'},
 {'role': 'user',
  'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '},
 {'role': 'user', 'content': 'Enter guess #1'},
 {'role': 'assistant', 'content': 'COVER'},
 {'role': 'user',
  'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word a

In [7]:
# Code is runnable but something is off
i = 0

while True:
    input_ids = tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=5,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    guess_str = tokenizer.decode(response, skip_special_tokens=True)

    guess_eval = game.turn(guess_str)
    user_response = p.explanatory_response(guess_eval, guess_str, i)
    conversation.extend([
        {'role': 'assistant', 'content': guess_str},
        {'role': 'user', 'content': user_response}
    ])
    if guess_eval == 'GGGGG':
        break
    elif i >= 4:
        conversation.append({'role': 'system', 'content': "5 guesses exhausted"})
        break
    else:
        i += 1

print(conversation)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

[{'role': 'system',
  'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)},
 {'role': 'system', 'content': 'Example #1:'},
 {'role': 'user', 'content': 'Enter guess #0'},
 {'role': 'assistant', 'content': 'BAGEL'},
 {'role': 'user',
  'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '},
 {'role': 'user', 'content': 'Enter guess #1'},
 {'role': 'assistant', 'content': 'COVER'},
 {'role': 'user',
  'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word a